In questa pagina viene riportata la porzione di codice python che realizza la seconda parte del task, vale a dire lettura (e fruizione) dei documenti json da parte di un Kafka Consumer che provvede al loro caricamento sulla collezione MongoDB *page_info* (facente sempre parte del DB *wikitrend*).

# Installazione librerie necessarie

In [ ]:
pip install kafka-python

# Collegamento a MongoDB

Nel chunk qui sotto, come da prassi, dapprima avviene il collegamento alla piattaforma *mongodbatlas*, successivamente viene inizializzata la nuova collezione di output, *page_info*, che ospiterà i documenti contenenti le info delle pagine wiki.

In [ ]:
from pymongo import MongoClient
client = MongoClient("")
# Select DataBase
db = client.get_database('wikitrend')
# Initialize output collection
info_pages = db.page_info

# Inizializzazione Kafka Consumer

Nel codice sottostante viene inizializzato un nuovo Kafka Consumer, il quale poi si iscrive al topic *trial*.

In [ ]:
from kafka import KafkaConsumer
import json

consumer = KafkaConsumer(
  bootstrap_servers=['kafka:9092'],
  auto_offset_reset='latest',
  value_deserializer=lambda v: json.loads(v.decode('utf-8')))

consumer.subscribe(['trial'])

# Lettura dei messaggi e caricamento su MongoDB

Nel codice sotto riportato il Consumer legge i messaggi che vengono inseriti nel topic *trial* (ovvero, i documenti json), in seguito provvede al loro caricamento nella collezione Mongo *info_pages*.

In [ ]:
for msg in consumer:
    data = msg.value
    info_pages.insert_one(data)